In [2]:
import numpy as np
import tensorflow as tf
import cv2
import keras
from keras import layers
from keras import optimizers
from keras.layers import Input, Lambda, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, AlphaDropout
from keras.layers import Conv2D, SpatialDropout2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.utils import plot_model
import keras.backend as K
from keras.applications import resnet, resnet_v2, vgg16, vgg19
from keras.datasets import mnist
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
# prepare data

(x_tr, y_tr), (x_test_norm, y_test_norm) = mnist.load_data()
x_tr = np.asarray([cv2.resize(a, (56, 56), interpolation = cv2.INTER_LINEAR) for a in x_tr])
x_test_norm = np.asarray([cv2.resize(a, (56, 56), interpolation = cv2.INTER_LINEAR) for a in x_test_norm])

x_tr = x_tr.reshape(60000, 56, 56, 1)
x_test_norm = x_test_norm.reshape(10000, 56, 56, 1)
y_tr = to_categorical(y_tr)
y_test_norm = to_categorical(y_test_norm)

In [12]:
# create model

a = Input(shape = (56, 56, 1))
x = Flatten()(a)
x = Dense(3136, activation = 'relu')(x)
x = BatchNormalization()(x)

x = Dense(10, activation = 'softmax')(x)

model = Model(inputs=a, outputs=x)

In [13]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 56, 56, 1)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 3136)              9837632   
_________________________________________________________________
batch_normalization_5 (Batch (None, 3136)              12544     
_________________________________________________________________
dense_10 (Dense)             (None, 10)                31370     
Total params: 9,881,546
Trainable params: 9,875,274
Non-trainable params: 6,272
_________________________________________________________________


In [19]:
optimizer = keras.optimizers.SGD(lr=0.0005, momentum = 0.9, nesterov = True)
model.compile(optimizer, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [20]:
model.fit(x_tr, y_tr, validation_data = (x_test_norm, y_test_norm), batch_size=32, epochs=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 14s 231us/step - loss: 0.0729 - accuracy: 0.9775 - val_loss: 0.1339 - val_accuracy: 0.9752


In [35]:
model.evaluate(x_test_norm, y_test_norm)

10000/10000 [==============================] - 1s 56us/step


[0.13392522302260623, 0.9751999974250793]

In [34]:
im = cv2.imread('mnist test number.png', cv2.IMREAD_GRAYSCALE)
im = cv2.resize(im, (56, 56), interpolation = cv2.INTER_LINEAR).reshape(1, 56, 56, 1)
out = np.argmax(model.predict(im))
print('The number is: ', out)

The number is:  0


In [21]:
im = cv2.imread('mnist test number.png')

In [36]:
model.save('models/FCN.h5')